In [1]:
import tweepy
import csv
import pandas as pd
####input your credentials here
consumer_key = 'Xnwinf3jwzaJx3GvKpWV0nTGj'
consumer_secret = 'bREfnZxQ45Kx60LdtrOKztNHmWAhdVAtALpxlssJ7TV26jvECg'
access_token = '1036313393114767360-05BDRZWgsbUYHkrrD2OOi0wrBsvEcp'
access_token_secret = 'eX6LQhih0UHIyKEJ7EBgL2KFrM58DSIe2sNMQkSjmhFC9'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####United Airlines
# Open/Create a file to append data
csvFile = open('ua.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="#unitedAIRLINES",count=100,
                           lang="en",
                           since="2017-04-03").items():
    print (tweet.created_at, tweet.text)
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

ModuleNotFoundError: No module named 'tweepy'